<a href="https://colab.research.google.com/github/SatyamRaj1/Bias-Detection/blob/main/BERT_Heirarchichal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.8 MB/s eta 0:00:00
  Installing build dependencies ... canceledERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line

In [ ]:
 import pandas as pd
 import numpy as np

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Data/ToxicBias")

In [ ]:
Train2 = pd.read_csv("toxicbias_train.csv")
Train2.head()

,comment_text,identity_attack_x,bias,target,category,rationale
0,We are still having their water put at risk in...,0.600000,bias,"whites,men","race,gender",prejudice against white men
1,This whole tragedy is the face and body and ph...,0.500000,bias,muslims,religion,prejudice against muslims
2,Black on black killings. Stop this madness ma...,0.706667,bias,blacks,race,prejudice against blacks
3,The world would be at peace with out islam.,0.833333,bias,muslims,religion,prejudice against islam
4,Another Muslim commits a terrorist act and dem...,0.500000,bias,muslims,religion,prejudice against muslims


In [ ]:
Train = pd.read_csv("toxicbias_train.csv", usecols = ['comment_text', 'bias', 'category'])
Test = pd.read_csv("toxicbias_test.csv", usecols = ['comment_text', 'bias', 'category'])
Val = pd.read_csv("toxicbias_val.csv", usecols = ['comment_text', 'bias', 'category'])

In [ ]:
Train['comment_text'][0]

'We are still having their water put at risk in the Dakotas for some white mans profit. When white people complained about the same issue the natives were just not important enough to care about their water supply. Some of you white folks refuse to acknowledge how much you benefit from the rape robbery torture and murder done in your benefit. Many of you are quite proud in your self righteous ignorance. Or maybe you just dont think those lives matter. Or they both may be true.'

In [ ]:
import re, string
import unicodedata
import html
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# # set of stopwords to be removed from text
# stop = set(stopwords.words('english'))

# # update stopwords to have punctuation too
# stop.update(list(string.punctuation))

stop = set(list(string.punctuation))
def clean_text(text):

    # Remove unwanted html characters
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
    'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
    '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
    ' @-@ ', '-').replace('\\', ' \\ ')
    text = re1.sub(' ', html.unescape(x1))

    # remove non-ascii characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

#     # strip html
#     soup = BeautifulSoup(text, 'html.parser')
#     text = soup.get_text()

    # remove between square brackets
    text = re.sub('\[[^]]*\]', '', text)

    # remove URLs
    text = re.sub(r'http\S+', '', text)

    # remove twitter tags
    text = text.replace("@", "")

    # remove hashtags
    text = text.replace("#", "")

    # remove all non-alphabetic characters
    text = re.sub(r'[^a-zA-Z ]', '', text)

    # remove stopwords from text
    final_text = []
    for word in text.split():
        if word.strip().lower() not in stop:
          final_text.append(word.strip().lower())

    text = " ".join(final_text)

    # lemmatize words
    # lemmatizer = WordNetLemmatizer()
    # text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    # text = " ".join([lemmatizer.lemmatize(word, pos = 'v') for word in text.split()])

    # replace all numbers with "num"
    text = re.sub("\d", "num", text)

    return text.lower()

In [ ]:
Train['comment_text'] = Train['comment_text'].apply(clean_text)
Test['comment_text'] = Test['comment_text'].apply(clean_text)
Val['comment_text'] = Val['comment_text'].apply(clean_text)

In [ ]:
result = [0]*6
for cat in Train['category']:
  c = 0
  c += 'lgbtq' in cat
  c += 'race' in cat
  c += 'religion' in cat
  c += 'gender' in cat
  c += 'political' in cat
  result[c] += 1
result, Train.shape

([868, 3021, 394, 41, 3, 0], (4327, 3))

In [ ]:
df_train = pd.DataFrame(list(zip(Train['comment_text'], Train['bias'])),columns =['text', 'label'])
df_test = pd.DataFrame(list(zip(Test['comment_text'], Test['bias'])),columns =['text', 'label'])
df_val = pd.DataFrame(list(zip(Val['comment_text'], Val['bias'])),columns =['text', 'label'])
df_train.head()

,text,label
0,we are still having their water put at risk in...,bias
1,this whole tragedy is the face and body and ph...,bias
2,black on black killings stop this madness man ...,bias
3,the world would be at peace with out islam,bias
4,another muslim commits a terrorist act and dem...,bias


In [ ]:
def changeOutput(data, text, column = 'label'):
  for i in range(data.shape[0]):
    data[column][i] = (0, 1)[text == data[column][i]]

In [ ]:
changeOutput(df_train,'bias')
changeOutput(df_test, 'bias')
changeOutput(df_val, 'bias')

In [ ]:
df_train.dtypes

text     object
label    object
dtype: object

In [ ]:
from bertopic import BERTopic
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import (set_seed,TrainingArguments,Trainer,AdamW,get_linear_schedule_with_warmup)
from sklearn.metrics import roc_auc_score

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import torch
from torch.utils.data import DataLoader

In [ ]:
class CustomDataset:

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.label = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            pad_to_max_length = True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'label': torch.tensor(float(self.label[index]), dtype=torch.float),
            'text' : text
        }

In [ ]:
epochs = 25
batch_size = 32
max_length = 128
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_labels = 2

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased',return_dict=True,output_hidden_states=True)
        self.l2 = torch.nn.Linear(768,128)
        self.l3 = torch.nn.Linear(128, n_labels)

    def forward(self, ids, mask, token_type_ids):
        output_1= self.l1(ids, attention_mask = mask)
        output_1 = output_1.hidden_states[-1][:,0,:]
        output_2 = self.l2(output_1)
        output_3 = self.l3(output_2)
        return output_3

model = BERTClass()
model = model.to(device)

In [ ]:
train_dataset = CustomDataset(df_train, tokenizer, max_length)
test_dataset = CustomDataset(df_test, tokenizer, max_length)
train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': batch_size,
                'shuffle': False,
                'num_workers': 0
                }

train_dataloader = DataLoader(train_dataset, **train_params)
test_dataloader = DataLoader(test_dataset, **test_params)

In [ ]:
def loss_fn(outputs, targets):
    crossEntropy = torch.nn.CrossEntropyLoss()
    return crossEntropy(outputs, targets)

In [ ]:
optimizer = torch.optim.AdamW(params =  model.parameters(), lr = 3e-5, eps = 1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 200, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import time

In [ ]:
def train_model(epoch):
    model.train()
    t0_epoch = time.time()
    total_loss = 0
    for _,data in enumerate(train_dataloader, 0):

        ids = data['ids'].to(device, dtype = torch.long)
        ids = ids.squeeze()
        mask = data['mask'].to(device, dtype = torch.long)
        mask = mask.squeeze()
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['label'].to(device, dtype = torch.long)

        outputs = model(ids, mask,token_type_ids)

        optimizer.zero_grad() #

        loss = loss_fn(outputs, targets)

        loss.backward() #

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) #

        total_loss += loss.item()

        optimizer.step() #

        scheduler.step() #

    avg_train_loss = total_loss / len(train_dataloader)
    time_elapsed = time.time() - t0_epoch
    print(f"{epoch + 1:^7} | {avg_train_loss:^12.6f} | {time_elapsed:^9.2f}")

In [ ]:
def test_model(epoch):
    model.eval()
    final_targets=[]
    final_outputs=[]
    val_texts = []
    with torch.no_grad():
        for _, data in enumerate(test_dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            ids = ids.squeeze()
            mask = data['mask'].to(device, dtype = torch.long)
            mask = mask.squeeze()
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['label']
            texts = data['text']

            outputs = model(ids, mask,token_type_ids)
            final_targets.extend(targets.numpy().tolist())
            final_outputs.extend(outputs.cpu().detach().numpy().tolist())
            val_texts.extend(texts)
    return final_outputs, final_targets, val_texts

In [ ]:
print("Start training...\n")
print ( f"{'Epoch':^7} | {'Train_Loss':^12} | {'Elapsed':^9}")
print("-"*35)
for epoch in range(epochs):
    train_model(epoch)

Start training...

 Epoch  |  Train_Loss  |  Elapsed 
-----------------------------------
   1    |   0.535341   |   81.72  
   2    |   0.441262   |   84.62  
   3    |   0.285453   |   85.47  
   4    |   0.134742   |   85.36  
   5    |   0.050076   |   85.35  
   6    |   0.040320   |   85.32  
   7    |   0.032751   |   85.57  
   8    |   0.015716   |   85.27  
   9    |   0.009700   |   85.35  
  10    |   0.007528   |   85.21  
  11    |   0.008942   |   85.56  
  12    |   0.002331   |   85.37  
  13    |   0.005758   |   85.31  
  14    |   0.001300   |   85.33  
  15    |   0.005909   |   85.32  
  16    |   0.002892   |   85.57  
  17    |   0.001936   |   85.33  
  18    |   0.002582   |   85.25  
  19    |   0.000803   |   85.27  
  20    |   0.000228   |   85.46  
  21    |   0.002158   |   85.33  
  22    |   0.003515   |   85.24  
  23    |   0.001856   |   85.27  
  24    |   0.000036   |   85.49  
  25    |   0.000034   |   85.26  


In [ ]:
outputs, targets, texts = test_model(epoch)

In [ ]:
import numpy as np
outputs = np.argmax(outputs, axis = 1)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['positive','negative']
y_true = targets
y_pred = outputs
print(classification_report(y_true, y_pred,target_names=target_names))
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

    positive       0.42      0.22      0.29       131
    negative       0.82      0.92      0.87       519

    accuracy                           0.78       650
   macro avg       0.62      0.57      0.58       650
weighted avg       0.74      0.78      0.75       650

              precision    recall  f1-score   support

         0.0       0.42      0.22      0.29       131
         1.0       0.82      0.92      0.87       519

    accuracy                           0.78       650
   macro avg       0.62      0.57      0.58       650
weighted avg       0.74      0.78      0.75       650



In [ ]:
torch.save(model.state_dict(), "model_sentiment.bin")